In [1]:
import pandas as pd
import numpy as np
import re
import datetime
now = datetime.datetime.now()
time = now.strftime("%d%m%y")

In [ ]:
pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.4 MB/s eta 0:00:00


In [1]:
!pip install unidecode


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## LOADING DATAFRAME

In [4]:
!mkdir output

In [5]:
import os
import re
import unidecode

# Get all files in the directory
files = os.listdir('/content')
# Define the regular expression pattern
#pattern = re.compile(r'^tuyensinh-.*\.xlsx$')
pattern2 = re.compile(r'^Báo cáo tiếp nhận.*\.xlsx$')

# Find and print matching files
for item in files:
    #if pattern.match(item):
        #file_name = item
        #print(file_name)
    if pattern2.match(item):
        file_name2 = item
        print(file_name2)

#path_thongtincu = f'/content/{file_name}'
path_thongtinmoi = f'/content/{file_name2}'
print(path_thongtinmoi)

Báo cáo tiếp nhận đại học - DUV7f3.xlsx
/content/Báo cáo tiếp nhận đại học - DUV7f3.xlsx


#### QUY VỀ TÊN KHOA; TẠO CÚ PHÁP CHUYỂN KHOẢN, REMOVE EXTRA SPACES

In [6]:
def assign_khoa(ma_nganh):
  if pd.isnull(ma_nganh):
    return None
  elif ('7480201' in ma_nganh
      or '7480103' in ma_nganh
      or '7480107' in ma_nganh):
    return 'Khoa Công nghệ thông tin'
  elif ('7510301' in ma_nganh
        or '7510205' in ma_nganh
        or '7510202' in ma_nganh
        or '7510103' in ma_nganh):
    return 'Khoa Kỹ thuật'
  elif ('7540101' in ma_nganh
        or '7510406' in ma_nganh
        or '7510401' in ma_nganh
        or '7420201' in ma_nganh
        or '7720301' in ma_nganh
        or '7720601' in ma_nganh):
    return 'Khoa Công nghệ'
  elif ('7320104' in ma_nganh
        or '7210403' in ma_nganh
        or '7210408' in ma_nganh):
    return 'Khoa Truyền thông - Thiết kế'
  elif ('7340101' in ma_nganh
        or '7810103' in ma_nganh
        or '7810201' in ma_nganh
        or '7510605' in ma_nganh):
    return 'Khoa Kinh tế quản trị'
  elif ('7340301' in ma_nganh
        or '7340201' in ma_nganh):
    return 'Khoa Kế toán - Tài chính'
  elif ('7220201' in ma_nganh
        or '7310608' in ma_nganh
        or '7220204' in ma_nganh):
    return 'Khoa Ngoại ngữ'
  else:
    return None

def remove_accent(text):
  # Check if text is a string before applying unidecode
  if isinstance(text, str):
    return unidecode.unidecode(text)
  # If not a string, convert to string first
  else:
    return unidecode.unidecode(str(text))

def unicode_hoten(df):
  hoten = (df['Họ'].apply(remove_accent) + ' '+ df['Tên'].apply(remove_accent)
           + ' ' + df['Điện Thoại']
           + ' ' + df['Ngày Sinh'].str.replace(r'/', '', regex=True)
           + ' ' + 'XETHB2025')
  return hoten

def remove_extra_spaces(text):
  if isinstance(text, str):
    return ' '.join(text.split())
  return text

In [7]:
def wrangling(path):
  # bước 1 đọc file excel
  df = pd.read_excel(path, dtype=str)
  # replace nan value by 0 value ở cột Mã hồ sơ
  # df['Mã hồ sơ'].fillna("0", inplace=True)
  # định dạng lại Họ và tên
  df['Họ'] = df['Họ'].str.title()
  df['Tên'] = df['Tên'].str.title()
  # drop cột STT
  df.drop('STT', axis=1, inplace=True)
  # điểm khu vực
  df['diem_khuvuc'] = df['Khu Vực theo THPT'].apply({'KV 1':0.75, 'KV 2-NT':0.5,
                                                  'KV 2':0.25}.get)
  df['diem_khuvuc'] = df['diem_khuvuc'].fillna(0)
  # asign Khoa for dataset
  df['Khoa_100'] = df['Mã ngành_100'].apply(assign_khoa)
  df['Khoa_200'] = df['Mã ngành_200'].apply(assign_khoa)
  df['Khoa_402'] = df['Mã ngành_402'].apply(assign_khoa)
  df['Khoa_thang'] = df['Mã ngành_tuyển thẳng'].apply(assign_khoa)
  # create new column dienthoai for ZNS
  df['dien_thoai_HS'] = '84' + df['Điện Thoại'].str[1:]
  df['dien_thoai_PH'] = '84' + df['Số điện thoại phụ huynh'].str[1:]
  # Cú pháp lệ phí 40.000
  df['cu_phap_40k'] = unicode_hoten(df)
  return df
df = wrangling(path_thongtinmoi)

## KIỂM TRA ĐĂNG KÝ THIẾU THÔNG TIN

In [8]:
lst1 = ['Họ', 'Tên', 'Giới Tính', 'Ngày Sinh', 'Điện Thoại',
       'Số CMCD/CCCD', 'Email', 'Nơi sinh', 'Dân tộc', 'Mã Tỉnh', 'Mã Huyện',
       'Địa chỉ liên lạc']
lst2 = ['Năm tốt nghiệp', 'Khu Vực theo THPT', 'Tỉnh THPT', 'Mã tỉnh THPT',
        'Mã trường THPT', 'Trường THPT']
lst3 = ['Mã ngành_200', 'Hình thức', 'Tổ hợp', 'Môn 1', 'Môn 2', 'Môn 3',
       'Tổng điểm']
lst4 = ['Học lực 12', 'Hạnh kiểm 12']
lst5 = ['Có học bạ', 'Bằng tốt nghiệp', 'Chứng nhận KQ thi TN',
        'Hình CMND/CCCD']
lst = lst1 + lst2 + lst3 + lst4 + lst5
thieu_thongtin = pd.DataFrame(index=df.index)
for item in lst1:
  for index, row in df.iterrows():
    if pd.isnull(row[item]):
      thieu_thongtin.loc[index, 'Thong_tin_thieu'] = f'[Thông tin thí sinh]'


thieu_thongtin['THPT'] = ''
for item in lst2:
  for index, row in df.iterrows():
    if pd.isnull(row[item]):
      thieu_thongtin.loc[index, 'THPT'] = f'[Quá trình học THPT]'

thieu_thongtin['NV'] = ''
for item in lst3:
  for index, row in df.iterrows():
    if pd.isnull(row[item]):
      thieu_thongtin.loc[index, 'NV'] = f'[Thông tin đăng ký nguyện vọng]'

thieu_thongtin['HLHK'] = ''
for item in lst4:
  for index, row in df.iterrows():
    if pd.isnull(row[item]):
      thieu_thongtin.loc[index, 'HLHK'] = f'[Học lực, hạnh kiểm]'

thieu_thongtin['scan'] = ''
for item in lst5:
  for index, row in df.iterrows():
    if row[item] == '0':
      thieu_thongtin.loc[index, 'scan'] = f'[File scan hồ sơ thí sinh]'

thieu_thongtin['cu_the'] = ''
for item in lst:
  for index, row in df.iterrows():
    if pd.isnull(row[item]):
      thieu_thongtin.loc[index, 'cu_the'] += f'[{item}] '
    elif row[item]=='0':
      thieu_thongtin.loc[index, 'cu_the'] += f'[{item}] '

thieu_thongtin.fillna("", inplace=True)

df['ZNS1_thieu'] = (thieu_thongtin['Thong_tin_thieu'] + ' '
                   + thieu_thongtin['THPT'] + ' '
                   + thieu_thongtin['NV'] + ' '
                   + thieu_thongtin['HLHK'] + ' '
                   + thieu_thongtin['scan'] + ' ')
df['ZNS1_thieu'] = df['ZNS1_thieu'].apply(remove_extra_spaces)

df['thieu_cu_the'] = thieu_thongtin['cu_the']
df['ZNS2_thieu'] = thieu_thongtin['scan']

#### THỐNG KÊ THEO NGÀY - THÁNG - NĂM

In [9]:
def thongke(df):
  # Theo ngày
  dangky = pd.DataFrame()
  dangky['Ngày đăng ký'] = pd.to_datetime(df['Ngày đăng ký'], format='%d/%m/%Y')
  dangky['Họ'] = df['Họ']
  dangky_days = dangky.groupby('Ngày đăng ký')['Họ'].count()
  # Theo tuần
  dangky_tuan = pd.DataFrame()
  dangky_tuan['week'] = dangky['Ngày đăng ký'].dt.isocalendar().week
  dangky_tuan['Họ'] = dangky['Họ']
  weeks = dangky_tuan.groupby('week')['Họ'].count()
  dangky_tuan = pd.DataFrame(weeks)
  dangky_tuan.reset_index(inplace=True)
  # Theo tháng
  dangky_thang = pd.DataFrame()
  dangky_thang['month'] = dangky['Ngày đăng ký'].dt.month
  dangky_thang['Họ'] = dangky['Họ']
  months = dangky_thang.groupby('month')['Họ'].count()
  dangky_thang = pd.DataFrame(months)
  dangky_thang.reset_index(inplace=True)
  return dangky_days, dangky_tuan, dangky_thang

dangky_days, dangky_tuan, dangky_thang = thongke(df)
with pd.ExcelWriter(f'output/TK-DWM-{time}.xlsx', engine='xlsxwriter') as writer:
    dangky_days.to_excel(writer, sheet_name='Tk Theo ngày')
    dangky_tuan.to_excel(writer, sheet_name='Tk Theo tuần')
    dangky_thang.to_excel(writer, sheet_name='Tk Theo tháng')

#### USING LOOKER STUDIO

In [10]:
looker = df[['Số CMCD/CCCD', 'Giới Tính', 'Dân tộc', 'Khu Vực theo THPT',
              'Tỉnh THPT', 'Trường THPT', 'Năm tốt nghiệp', 'Ngày đăng ký',
              'Tên ngành_100', 'Tên ngành_200', 'Tên ngành_402',
              'Mã ngành_tuyển thẳng', 'Khoa_100', 'Khoa_200', 'Khoa_402',
              'Khoa_thang', 'dien_thoai_PH', 'Hoàn thành', 'Đã kiểm tra',
              'Có học bạ', 'Bằng tốt nghiệp', 'Chứng nhận KQ thi TN','ZNS1_thieu']]
looker['Tên ngành_tuyển thẳng'] = looker['Mã ngành_tuyển thẳng'].apply({'7720301': 'Điều dưỡng',
                                          '7220201': 'Ngôn ngữ Anh',
                                          '7340201': 'Tài chính - Ngân hàng',
                                          '7510605': 'Logistics và quản lý chuỗi cung ứng',
                                          '7220204': 'Ngôn ngữ Trung Quốc',
                                          '7480201': 'Công nghệ thông tin',
                                          '7480107': 'Trí tuệ nhân tạo',
                                          '7510301': 'Công nghệ kỹ thuật điện, điện tử',
                                          '7510202': 'Công nghệ chế tạo máy ',
                                          '7480103': 'Kỹ thuật phần mềm',
                                          '7810201': 'Quản trị khách sạn',
                                          '7320104': 'Truyền thông đa phương tiện ',
                                          '7340101': 'Quản trị kinh doanh',
                                          '7720601': 'Kỹ thuật xét nghiệm y học',
                                          '7210403': 'Thiết kế đồ họa ',
                                          '7420201': 'Công nghệ sinh học',
                                          '7540101': 'Công nghệ thực phẩm',
                                          '7340301': 'Kế toán ',
                                          '7510205': 'Công nghệ kỹ thuật ô tô',
                                          '7210408': 'Nghệ thuật số',
                                          '7510406': 'Công nghệ kỹ thuật môi trường',
                                          '7510401': 'Công nghệ kỹ thuật hóa học',
                                          '7810103': 'Quản trị dịch vụ du lịch và lữ hành',
                                          '7310608': 'Đông phương học',
                                          '7510103': 'Công nghệ kỹ thuật xây dựng'
                                          }.get)
looker.to_csv(f'output/studio-looker-{time}.csv', index= False)

<ipython-input-10-45abc8db8ad7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  looker['Tên ngành_tuyển thẳng'] = looker['Mã ngành_tuyển thẳng'].apply({'7720301': 'Điều dưỡng',


In [11]:
df.columns

Index(['Mã hồ sơ', 'Họ', 'Tên', 'Giới Tính', 'Ngày Sinh', 'Điện Thoại',
       'Số CMCD/CCCD', 'Email', 'Nơi sinh', 'Dân tộc', 'Năm tốt nghiệp',
       'Khu Vực theo THPT', 'Mã Tỉnh', 'Mã Huyện', 'Tỉnh THPT', 'Mã tỉnh THPT',
       'Mã trường THPT', 'Trường THPT', 'Địa chỉ liên lạc', 'Mã ngành_100',
       'Tên ngành_100', 'Mã ngành_200', 'Tên ngành_200', 'Mã ngành_402',
       'Tên ngành_402', 'Hình thức', 'Tổ hợp', 'Môn 1', 'Môn 2', 'Môn 3',
       'Tổng điểm', 'Mã ngành_tuyển thẳng', 'Học lực 12', 'Hạnh kiểm 12',
       'Ngày đăng ký', 'Hoàn thành', 'Có học bạ', 'Bằng tốt nghiệp',
       'Chứng nhận KQ thi TN', 'Hình CMND/CCCD', 'Đã kiểm tra',
       'Số điện thoại phụ huynh', 'Ghi chú', 'Tên người giới thiệu',
       'Số điện thoại người giới thiệu', 'Câu hỏi khảo sát 1', 'diem_khuvuc',
       'Khoa_100', 'Khoa_200', 'Khoa_402', 'Khoa_thang', 'dien_thoai_HS',
       'dien_thoai_PH', 'cu_phap_40k', 'ZNS1_thieu', 'thieu_cu_the',
       'ZNS2_thieu'],
      dtype='object')

## LƯU DỮ LIỆU

In [12]:
df.to_excel(f'output/export-{time}.xlsx')

In [13]:
!zip -r /content/{time}.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/export-270525.xlsx (deflated 3%)
  adding: content/output/TK-DWM-270525.xlsx (deflated 11%)
  adding: content/output/studio-looker-270525.csv (deflated 89%)


In [14]:
df.columns

Index(['Mã hồ sơ', 'Họ', 'Tên', 'Giới Tính', 'Ngày Sinh', 'Điện Thoại',
       'Số CMCD/CCCD', 'Email', 'Nơi sinh', 'Dân tộc', 'Năm tốt nghiệp',
       'Khu Vực theo THPT', 'Mã Tỉnh', 'Mã Huyện', 'Tỉnh THPT', 'Mã tỉnh THPT',
       'Mã trường THPT', 'Trường THPT', 'Địa chỉ liên lạc', 'Mã ngành_100',
       'Tên ngành_100', 'Mã ngành_200', 'Tên ngành_200', 'Mã ngành_402',
       'Tên ngành_402', 'Hình thức', 'Tổ hợp', 'Môn 1', 'Môn 2', 'Môn 3',
       'Tổng điểm', 'Mã ngành_tuyển thẳng', 'Học lực 12', 'Hạnh kiểm 12',
       'Ngày đăng ký', 'Hoàn thành', 'Có học bạ', 'Bằng tốt nghiệp',
       'Chứng nhận KQ thi TN', 'Hình CMND/CCCD', 'Đã kiểm tra',
       'Số điện thoại phụ huynh', 'Ghi chú', 'Tên người giới thiệu',
       'Số điện thoại người giới thiệu', 'Câu hỏi khảo sát 1', 'diem_khuvuc',
       'Khoa_100', 'Khoa_200', 'Khoa_402', 'Khoa_thang', 'dien_thoai_HS',
       'dien_thoai_PH', 'cu_phap_40k', 'ZNS1_thieu', 'thieu_cu_the',
       'ZNS2_thieu'],
      dtype='object')